In [1]:
import raphtory as rp

lombardia_gasoline = rp.Graph.load_from_file("../network/same_price/same_price_lombardia_stations_gasoline")
lombardia_diesel = rp.Graph.load_from_file("../network/same_price/same_price_lombardia_stations_diesel")

In [5]:
import networkx as nx

distances_network = {}

distances = [1, 2, 3, 4, 5]

for km in distances:
    distances_network[km] = nx.read_gexf(f"../network/distance/{km}_km_distance_network_Lombardia.gexf")

In [10]:
for l in distances_network:
    print(f"Network for {l} km distance loaded with {len(distances_network[l].nodes())} nodes and {len(distances_network[l].edges())} edges.")

Network for 1 km distance loaded with 3131 nodes and 2371 edges.
Network for 2 km distance loaded with 3131 nodes and 7369 edges.
Network for 3 km distance loaded with 3131 nodes and 14872 edges.
Network for 4 km distance loaded with 3131 nodes and 25021 edges.
Network for 5 km distance loaded with 3131 nodes and 37177 edges.


In [ ]:
import datetime

def instant_matching(distances_network, price_network, start_date, end_date, region_name, fuel_type):

    actual_date = start_date

    for km in distances_network:

        for node in distances_network[km].nodes:

            fixed_node = price_network.node(node)

            for neighbor in distances_network[km].neighbors(node):

                if 'istant_matching' not in distances_network[km][node][neighbor]:
                    neighbor_node = price_network.node(neighbor)

                    matching_days = 0
                    total_days = 0

                    actual_date = start_date + datetime.timedelta(days=1)
                    while(actual_date <= end_date):

                        try:

                            change_fixed = float(fixed_node.at(actual_date).properties["prezzo"]) - float(fixed_node.at(actual_date - datetime.timedelta(days=1)).properties["prezzo"])
                            change_neighbor = float(neighbor_node.at(actual_date).properties["prezzo"]) - float(neighbor_node.at(actual_date - datetime.timedelta(days=1)).properties["prezzo"])

                            total_days += 1
                            if change_fixed * change_neighbor > 0:
                                #print(node, neighbor, actual_date, change_fixed,change_neighbor)
                                matching_days += 1
                            elif change_fixed == 0 and change_neighbor == 0:
                                #print(node, neighbor, actual_date, change_fixed,change_neighbor)
                                matching_days += 1
                                
                        except:
                            pass

                        actual_date += datetime.timedelta(days=1)

                    if total_days == 0:
                        distances_network[km][node][neighbor]["istant_matching"] = 0
                    else:
                        distances_network[km][node][neighbor]["istant_matching"] = round((matching_days / total_days)*100,2)
    
    for km in distances:
        file_name = f"../network/price_matching/{km}_km_price_matching_network_{region_name}_{fuel_type}.gexf"
        nx.write_gexf(distances_network[km], file_name)
    
    return distances_network


price_matching_network_gasoline = instant_matching(distances_network, lombardia_gasoline, datetime.datetime(2025, 4, 1), datetime.datetime(2025, 6, 30), "Lombardia", "gasoline")
price_matching_network_diesel = instant_matching(distances_network, lombardia_diesel, datetime.datetime(2025, 4, 1), datetime.datetime(2025, 6, 30), "Lombardia", "diesel")      